## Подготовка данных для занятия

In [26]:
import re

import pandas as pd
import pymorphy2


text_df = pd.read_csv(r"C:\Users\User\Desktop\Новая папка\19_data\data-2\content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==7].sort_values(by='dummy', ascending=False).head(5)

,doc_id,word,dummy
731,7,и,9
761,7,на,5
707,7,в,5
775,7,они,4
760,7,молодая,2


# 10.3  Домашняя работа

10.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [45]:
# -- ВАШ КОД ЗДЕСЬ --
doc_freq = pd.DataFrame()
word_count_df['dummy1'] = 1
amount = word_count_df.groupby('word').agg({'dummy1':sum}).sort_values(by='dummy1',ascending = False)
doc_freq['word_count'] = amount['dummy1']
doc_freq['word_count_pers'] = doc_freq['word_count']/8*100
doc_freq


,word_count,word_count_pers
word,,
и,8,100.0
в,8,100.0
на,7,87.5
с,7,87.5
он,6,75.0
...,...,...
леди,1,12.5
лестрейд,1,12.5
личико,1,12.5


10.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [90]:
# -- ВАШ КОД ЗДЕСЬ --
raw_text = text_df.description.values[3]
# регулярка - её нужно поправить
reg_expr = r'[А-Я]{1}\w*.[А-Я]{1}\w*'
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

Стивена Фрая
Джоном Дженксом
У Теда
Теперь Тед
Вскоре Тед


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [120]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор','расправа'],
    'сказка': ['приключения', 'милый', 'семейный','ёлка'],
    'детектив': ['тайна', 'разгадать', 'загадочный','криминальный','рискованный'], 
    'триллер': ['ужас', 'зловещий', 'нерв','рискованный','кровь']
}

nested_genres =[[(i, j) for j in genre_dict[i]] for i in genre_dict]
# переводим из словаря к более удобному виду
flatten_genres = list(itertools.chain(*nested_genres))
# создаём DataFrame
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'character_word'])


# -- ВАШ КОД ЗДЕСЬ --
genres_df.columns = ['genre', 'word']

genre_dict1= genres_df.merge(word_count_df, how='inner')

genre_dict2 = genre_dict1.drop(genre_dict1.columns[[1,3, 4]], axis='columns')
genre_dict2

,genre,doc_id
0,комедия,3
1,комедия,4
2,комедия,4
3,мелодрама,2
4,мелодрама,2
5,мелодрама,5
6,сказка,1
7,сказка,1
8,детектив,0
9,детектив,0
